
**Hull Tactical Market Prediction Competition
Portfolio Optimization Solution with Multiple Strategies**


### Acknowledging Kenshiro75 for the initial inspiration, this notebook was cloned from their notebook and modified.

The Hull Tactical Market Prediction competition challenges participants to predict S&P 500 excess returns while managing volatility constraints. The goal is to develop a trading strategy that outperforms buy-and-hold while maintaining volatility within 120% of the market's volatility.

## What is Portfolio Optimization?

**Portfolio Optimization** is the process of mathematically determining the optimal **position** (or allocation) in the risky asset (the market, S&P 500 excess returns) versus the risk-free asset (cash). The goal is to maximize a specific utility function or financial metric, such as the **Adjusted Sharpe Ratio**, subject to predefined constraints on the allocation size and risk.

The notebook defines the position as the allocation to the risky asset, where:
* 0.0 MIN POSITION represents 0% in the market (i.e., 100% in the risk-free asset).
* 1.0 represents 100% in the market (market neutral).
* 2.0 MAX POSITION represents 200% in the market (a leveraged, aggressive position).

The overall portfolio return is calculated based on this position:
Strategy Return = (Risk-Free Rate * (1 - Position)) + (Position * Market Return)

The optimization itself is performed using numerical methods like **L-BFGS-B** (for Mean-Variance) and **SLSQP** (for constrained and Sharpe maximization problems) via the `scipy.optimize.minimize` function.




## Role in the Hull Tactical Market Prediction Competition

The central role of portfolio optimization in this competition is to achieve a trading strategy that **outperforms a simple buy-and-hold strategy** while adhering to a critical risk constraint:

### 1. Primary Objective: Maximizing the Adjusted Sharpe Ratio
The ultimate goal of the competition is to develop a strategy that yields the highest **Adjusted Sharpe Ratio**. The strategies (Mean-Variance, Minimum Variance, Sharpe Maximization, Risk Parity, CAPM Alpha Signal) all aim to find a series of daily positions that maximize this specific metric. The strategy that produces the highest Adjusted Sharpe Ratio on the historical data is declared the **Winner**.

### 2. Managing Volatility and Returns (The Adjusted Sharpe Penalties)
The Adjusted Sharpe Ratio is the competition's core evaluation metric because it heavily **penalizes** strategies that violate two key constraints:

* **Excess Volatility Penalty**: This penalty is applied if the strategy's volatility is greater than **120% of the market's volatility**. 
* **Return Penalty**: This penalty is applied if the strategy's geometric mean excess return **underperforms the market's return**. 
By optimizing directly for this metric (as in the **Sharpe Maximization** strategy), the portfolio optimization implicitly manages these constraints to secure the highest possible rank.

The optimization process uses a **lookback window of 180 days** of historical data for all calculations, simulating the rolling window approach typically used in live trading. The final winning strategy is chosen as the one that yields the highest **Adjusted Sharpe Ratio** during this historical evaluation.

**Multiple Strategy Implementation**

Instead of a single optimization approach, we implement 6 distinct portfolio optimization strategies:

* Mean-Variance Optimization (Markowitz)
* Minimum Variance Portfolio
* Sharpe Ratio Maximization
* Risk Parity (Inverse Volatility) Allocation
* CAPM Alpha-based Signal Strategy
* Sortino Ratio Maximization

In [1]:
import numpy as np
import pandas as pd 
import polars as pl
import hashlib
from scipy.optimize import minimize, Bounds
from scipy.stats import skew, kurtosis
import warnings
import os
import kaggle_evaluation.default_inference_server 
warnings.filterwarnings('ignore')


In [2]:
# ============================================================================
# CONSTANTS AND CONFIGURATION
# ============================================================================

# Position represents the allocation to the risky asset (the market)
MIN_POSITION = 0.0  # Minimum investment (0% in market, 100% in risk-free) - Defensive/Conservative
MAX_POSITION = 2.0  # Maximum investment (200% in market - leveraged) - Aggressive/Leveraged
TRADING_DAYS_PER_YEAR = 252 # Standard assumption for annualized calculations
LOOKBACK_WINDOW = 180  # Days of historical data to use for portfolio optimization (rolling window)
EPS = 1e-10  # Small epsilon to avoid division by zero


The `calculate_adjusted_sharpe` function, is the core evaluation metric for the competition. It takes a series of portfolio positions and the corresponding historical market data, calculates the strategy's performance, and then adjusts the standard Sharpe Ratio by applying penalties for high volatility and underperforming the market.

In [3]:
# ===================================
# LOADING DATA
# ===================================

# Load training data
print("Loading training data...")
train_ = pd.read_csv('/kaggle/input/hull-tactical-market-prediction/train.csv')
test_ = pd.read_csv('/kaggle/input/hull-tactical-market-prediction/test.csv')


Loading training data...


#### 💡💡💡Dimistifying best score in leaderboard. ####

If you check your data, test set is included in the train set. So the best score in the public leaderboard (17.396) is coming from this effect. Version 2/4 of the notebook shows equal result between my private leaderboard and public leaderboard. Version 5 shows difference between them.
Credits to Luca Massaron


In [4]:
# ===========================================
# CHECK DATA
# ===========================================

In [5]:
# Use only common columns for comparison
common_cols = list(set(train_.columns) & set(test_.columns))
train_subset = train_[common_cols].copy()
test_subset = test_[common_cols].copy()

# Create hash for each row for efficient comparison
train_subset['_row_hash'] = pd.util.hash_pandas_object(train_subset, index=False)
test_subset['_row_hash'] = pd.util.hash_pandas_object(test_subset, index=False)
    
# Find overlapping rows
overlapping_hashes = set(test_subset['_row_hash']) & set(train_subset['_row_hash'])
test_only_hashes = set(test_subset['_row_hash']) - set(train_subset['_row_hash'])
results = {}    
results['exact_row_match'] = {
        'overlap_count': len(overlapping_hashes),
        'test_only_count': len(test_only_hashes),
        'overlap_percentage': (len(overlapping_hashes) / len(test_subset) * 100) if len(test_subset) > 0 else 0,
        'is_test_subset': len(test_only_hashes) == 0
}

results

{'exact_row_match': {'overlap_count': 10,
  'test_only_count': 0,
  'overlap_percentage': 100.0,
  'is_test_subset': True}}

In [6]:
# Train set distilled from test set
train=train_.iloc[:-10]


In [7]:
# ============================================================================
# EVALUATION METRIC (ADJUSTED SHARPE RATIO)
# ============================================================================

def calculate_adjusted_sharpe(solution_df, predictions):
    """
    Calculate the competition's adjusted Sharpe ratio.
    
    This metric is crucial as the strategies optimize directly for this score.
    It penalizes:
    1. Strategies with volatility > 120% of market volatility (Excess Volatility Penalty)
    2. Strategies that underperform the market return (Return Penalty)
    
    Parameters:
    -----------
    solution_df : pd.DataFrame
        Contains 'forward_returns' (market return) and 'risk_free_rate' (benchmark/cash return)
    predictions : array-like
        Position sizes (0 to 2) - the output of the optimization strategies
    
    Returns:
    --------
    float : Adjusted Sharpe ratio
    """
    solution = solution_df.copy()
    solution['position'] = predictions
    
    # Calculate the portfolio's realized daily returns
    # Strategy Return = (Risk-Free Rate * Uninvested Portion) + (Position * Market Return)
    solution['strategy_returns'] = (
        solution['risk_free_rate'] * (1 - solution['position']) +
        solution['position'] * solution['forward_returns']
    )
    
    # Strategy performance metrics
    strategy_excess = solution['strategy_returns'] - solution['risk_free_rate']
    strategy_cum_excess = (1 + strategy_excess).prod()
    
    # Severe penalty for cumulative loss
    if strategy_cum_excess <= 0:
        return -1000.0
    
    # Calculate annualized geometric mean excess return
    strategy_mean_excess = strategy_cum_excess ** (1 / len(solution)) - 1
    strategy_std = solution['strategy_returns'].std()
    
    if strategy_std == 0:
        return 0.0
    
    # Standard Sharpe ratio calculation (annualized)
    sharpe = strategy_mean_excess / strategy_std * np.sqrt(TRADING_DAYS_PER_YEAR)
    strategy_volatility = strategy_std * np.sqrt(TRADING_DAYS_PER_YEAR) * 100 # Annualized volatility in %
    
    # Market performance metrics for benchmarking
    market_excess = solution['forward_returns'] - solution['risk_free_rate']
    market_cum_excess = (1 + market_excess).prod()
    
    if market_cum_excess <= 0:
        market_mean_excess = -1.0
    else:
        market_mean_excess = market_cum_excess ** (1 / len(solution)) - 1
    
    market_std = solution['forward_returns'].std()
    market_volatility = market_std * np.sqrt(TRADING_DAYS_PER_YEAR) * 100 # Annualized market volatility in %
    
    # Apply penalties
    # Volatility penalty: Kicks in when strategy vol > 120% of market vol
    excess_vol_penalty = 1.0
    if market_volatility > 0:
        vol_ratio = strategy_volatility / market_volatility
        if vol_ratio > 1.2:
            # Multiplicative penalty based on how much the ratio exceeds 1.2
            excess_vol_penalty = 1 + (vol_ratio - 1.2)
    
    # Return penalty: Penalizes underperformance vs market
    return_gap = max(0, (market_mean_excess - strategy_mean_excess) * 100 * TRADING_DAYS_PER_YEAR) # Annualized return difference
    # Quadratic penalty: gap^2 / 100
    return_penalty = 1 + (return_gap ** 2) / 100
    
    # Final adjusted Sharpe: standard Sharpe / (Volatility Penalty * Return Penalty)
    adjusted_sharpe = sharpe / (excess_vol_penalty * return_penalty)
    
    return float(min(adjusted_sharpe, 1_000_000)) # Clip maximum value


In [8]:
# ============================================================================
# SORTINO RATIO
# ============================================================================

def calculate_sortino_ratio(solution_df, predictions):
    """
    Calculate the Sortino Ratio.
    MAR (Minimum Acceptable Return) is set to the risk-free rate.
    """
    solution = solution_df.copy()
    solution['position'] = predictions
    
    # Calculate the portfolio's realized daily returns
    solution['strategy_returns'] = (
        solution['risk_free_rate'] * (1 - solution['position']) +
        solution['position'] * solution['forward_returns']
    )
    
    # Excess return over MAR (Risk-Free Rate)
    strategy_excess = solution['strategy_returns'] - solution['risk_free_rate']
    strategy_cum_excess = (1 + strategy_excess).prod()

    if strategy_cum_excess <= 0:
        return -1000.0
    
    # Annualized geometric mean excess return (Numerator)
    strategy_mean_excess = strategy_cum_excess ** (1 / len(solution)) - 1
    
    # Downside Deviation (Denominator)
    # Only consider returns below the risk-free rate (i.e., excess returns < 0)
    downside_returns = strategy_excess[strategy_excess < 0]
    
    # If there are no downside returns, downside deviation is 0.
    if len(downside_returns) == 0:
        # Return a large number to favor this position set
        return 1000.0

    # Downside deviation (annualized)
    downside_std = np.std(downside_returns)
    downside_deviation = downside_std * np.sqrt(TRADING_DAYS_PER_YEAR)
    
    if downside_deviation == 0:
        return 1000.0
    
    # Sortino Ratio calculation (annualized)
    # The numerator (strategy_mean_excess) is already annualized geometrically
    sortino = strategy_mean_excess / (downside_deviation / np.sqrt(TRADING_DAYS_PER_YEAR)) # Annualized Excess Return / Downside Volatility
    
    return float(min(sortino, 1_000_000))



The `create_features`, which performs feature engineering by generating time-series characteristics from the raw return data. These are typical technical analysis indicators that could be used as inputs for a more complex machine learning model.

The features are calculated using a rolling window approach to capture short-term, localized market dynamics.

**Feature Engineering**

Added market dynamics features:

* Rolling statistics (mean, standard deviation, skewness, kurtosis)
* Momentum indicators (5-day, 20-day)
* Volatility regime identification

In [9]:
# ============================================================================
# FEATURE ENGINEERING
# ============================================================================

def create_features(data):
    """
    Create additional features for portfolio optimization.
    
    These features capture time-series characteristics which could be used 
    by a more complex ML model to predict optimal positions, though they 
    are not explicitly used by the current simple optimization functions.
    They represent typical technical analysis indicators.
    """
    df = data.copy()
    
    # Basic returns if not present
    if 'returns' not in df.columns and 'forward_returns' in df.columns:
        df['returns'] = df['forward_returns']
    
    # Rolling window statistics (20-day = ~1 month)
    window = min(20, len(df) // 4)  # Adaptive window size to ensure sufficient data
    
    if len(df) >= window:
        # Volatility and central tendency
        df['rolling_mean'] = df['returns'].rolling(window).mean()
        df['rolling_std'] = df['returns'].rolling(window).std()
        # Higher-order moments (risk-related)
        df['rolling_skew'] = df['returns'].rolling(window).skew()
        df['rolling_kurt'] = df['returns'].rolling(window).apply(lambda x: kurtosis(x))
        
        # Momentum indicators (short and medium term)
        df['momentum_5'] = df['returns'].rolling(5).mean()
        df['momentum_20'] = df['returns'].rolling(20).mean() if len(df) >= 20 else np.nan
        
        # Volatility regime indicator
        df['vol_regime'] = df['rolling_std'] / df['rolling_std'].mean()
    
    # Fill NaN values (common in rolling calculations at the start of the series) with 0
    df = df.fillna(0)
    
    return df


In [10]:
# ============================================================================
# PORTFOLIO OPTIMIZATION STRATEGIES
# ============================================================================
# Each strategy uses scipy.optimize.minimize to find the optimal 'positions' array
# for the given lookback window, based on a specific financial objective function.


**Mean-Variance Optimization (Markowitz Portfolio Theory)**

Objective Function:

Maximize: E[R] - λ * Var(R)

Where:

E[R] = Expected return
Var(R) = Variance of returns
λ = Risk aversion parameter (set to 2.0)

***Characteristics:***

* Balances return and risk explicitly
* Risk aversion parameter can be tuned
* Classic approach with strong theoretical foundation
* Works well in stable market conditions

***When it works best:***

* Markets with clear risk-return tradeoffs
* When historical patterns are predictive
* Moderate volatility environments

In [11]:
# Strategy 1: Mean-Variance Optimization (Markowitz)
def mean_variance_optimization(data, target_return=None):
    """
    Classic Markowitz mean-variance optimization (MVO).
    
    The objective function maximizes a 'utility' function:
    Utility = Expected Return - (Risk Aversion * Variance)
    """
    n_days = len(data)
    
    def objective(positions):
        # The positions array is the variable being optimized
        pos = np.clip(positions, MIN_POSITION, MAX_POSITION)
        
        # Calculate portfolio returns based on positions
        portfolio_returns = (
            data['risk_free_rate'].values * (1 - pos) +
            pos * data['forward_returns'].values
        )
        
        # Mean and variance of the portfolio returns
        mean_return = np.mean(portfolio_returns)
        variance = np.var(portfolio_returns)
        
        # Risk aversion parameter (lambda). A higher value means the optimizer 
        # heavily penalizes variance. Set to 2.0 (arbitrary choice).
        risk_aversion = 2.0
        
        # Maximize: mean_return - risk_aversion * variance. 
        # Since 'minimize' is used, we negate the objective function.
        return -(mean_return - risk_aversion * variance) * TRADING_DAYS_PER_YEAR
    
    # Initial guess: equal weight (100% market allocation)
    x0 = np.full(n_days, 1.0)
    
    # Bounds: ensures positions stay within [MIN_POSITION, MAX_POSITION]
    bounds = Bounds(MIN_POSITION, MAX_POSITION)
    
    # Optimization using L-BFGS-B (a quasi-Newton method suitable for bounded problems)
    result = minimize(
        objective,
        x0,
        method='L-BFGS-B',
        bounds=bounds,
        options={'maxiter': 500, 'ftol': 1e-8}
    )
    
    return np.clip(result.x, MIN_POSITION, MAX_POSITION)


**Minimum Variance Portfolio**

Objective Function:

Minimize: Var(R)
Subject to: E[R] >= minimum_return

***Characteristics:***

* Defensive strategy focusing on risk minimization
* Includes return constraint to avoid trivial solutions
* Tends to produce lower but more stable returns
* Good for risk-averse investors

***When it works best:***

* High volatility periods
* Market downturns
* When capital preservation is priority

In [12]:
# Strategy 2: Minimum Variance Portfolio
def minimum_variance_optimization(data):
    """
    Minimize portfolio variance subject to a minimum return constraint.
    
    This strategy is defensive, focusing solely on risk reduction while 
    ensuring a basic level of performance above the risk-free rate.
    """
    n_days = len(data)
    
    def objective(positions):
        pos = np.clip(positions, MIN_POSITION, MAX_POSITION)
        
        portfolio_returns = (
            data['risk_free_rate'].values * (1 - pos) +
            pos * data['forward_returns'].values
        )
        
        # Objective: Minimize variance
        return np.var(portfolio_returns)
    
    # Start with a low-risk position guess (30% market allocation)
    x0 = np.full(n_days, 0.3)
    
    bounds = Bounds(MIN_POSITION, MAX_POSITION)
    
    # Define the minimum return constraint
    # Target: At least 50% above the historical mean risk-free rate
    min_return = data['risk_free_rate'].mean() * 1.5
    
    def return_constraint(positions):
        pos = np.clip(positions, MIN_POSITION, MAX_POSITION)
        portfolio_returns = (
            data['risk_free_rate'].values * (1 - pos) +
            pos * data['forward_returns'].values
        )
        # Constraint: Mean Portfolio Return - Min Target Return >= 0
        return np.mean(portfolio_returns) - min_return
    
    # Constraints list for SLSQP
    constraints = [{'type': 'ineq', 'fun': return_constraint}] # 'ineq' means inequality constraint (fun(x) >= 0)
    
    # Optimization using SLSQP (Sequential Least Squares Programming, good for constraints)
    result = minimize(
        objective,
        x0,
        method='SLSQP',
        bounds=bounds,
        constraints=constraints,
        options={'maxiter': 500, 'ftol': 1e-8}
    )
    
    return np.clip(result.x, MIN_POSITION, MAX_POSITION)



**Sharpe Ratio Maximization**

Objective Function:

Maximize: (E[R] - Rf) / σ(R)

With penalties for:
- Volatility > 120% of market
- Underperformance vs market

***Characteristics:***

* Directly optimizes the competition metric
* Accounts for both returns and risk
* Includes competition-specific penalties
* Most aligned with evaluation criteria

***When it works best:***

* When you need balanced risk-adjusted returns
* Competition scenarios with specific metrics
* Markets where relative performance matters

In [13]:
# Strategy 3: Sharpe Ratio Maximization
def sharpe_ratio_optimization(data):
    """
    Directly maximize the Adjusted Sharpe Ratio metric defined by the competition.
    
    This is the most direct and aggressive strategy.
    It uses multiple random starting points (x0) to increase the chance of 
    finding the global maximum for the non-linear objective function.
    """
    n_days = len(data)
    
    def objective(positions):
        return -calculate_adjusted_sharpe(data, positions)
    
    best_result = None
    best_score = float('-inf')
    
    # Test multiple initial guesses to avoid getting stuck in local minima/maxima
    initial_guesses = [
        np.full(n_days, 0.5),  # Moderate
        np.full(n_days, 1.0),  # Market neutral
        np.random.uniform(0.3, 0.7, n_days),  # Random moderate
        np.linspace(0.2, 0.8, n_days),  # Gradual increase
    ]
    
    bounds = Bounds(MIN_POSITION, MAX_POSITION)
    
    for x0 in initial_guesses:
        try:
            # Optimization using SLSQP (suitable for non-smooth objectives like Sharpe)
            result = minimize(
                objective,
                x0,
                method='SLSQP',
                bounds=bounds,
                options={'maxiter': 1000, 'ftol': 1e-8}
            )
            
            # Check if this run is successful and better than the current best
            if result.success and -result.fun > best_score:
                best_score = -result.fun
                best_result = result
        except:
            continue
    
    if best_result is not None:
        return np.clip(best_result.x, MIN_POSITION, MAX_POSITION)
    else:
        # Fallback to a conservative position if all optimizations fail
        return np.full(n_days, 0.5)




**Risk Parity (Inverse Volatility) Allocation**

Objective Function:

Minimize the variance contribution from each component to achieve equal risk contribution across all assets.

∑(σi−σp/n)^2

where σi is the risk contribution of asset, σp is the total portfolio risk, and n is the number of assets.


***Characteristics:***
* Focuses purely on risk rather than maximizing return or Sharpe Ratio.
* The core principle is to maintain a constant level of risk (or equal risk contribution) over time.
* Position size is inversely proportional to the asset's risk (volatility), meaning the allocation automatically decreases when risk spikes.
* It does not require estimation of expected returns (unlike Mean-Variance), making it more robust against common estimation errors.
* The allocation is calculated via a simple, explicit formula, not an iterative numerical solver.

***When it works best:***
* High Volatility Environments: The strategy is defensive, automatically reducing market exposure when volatility rises, leading to stability.
* Regime Shifts and Market Crises: It is robust because it relies only on the relatively stable estimation of volatility, not the volatile estimation of mean returns.
* Risk Management: When the primary goal is capital preservation and portfolio stability over chasing maximum returns.

In [14]:
# Strategy 4: Risk Parity (Inverse Volatility) Allocation 
def risk_parity_optimization(data):
    """
    Implements a simple single-asset Risk Parity (RP) allocation based on Inverse Volatility.
    
    This strategy sets the position inversely proportional to the asset's rolling volatility
    to maintain a constant risk contribution over time. 
    """
    # Use a 20-day rolling window for volatility calculation
    window = min(20, len(data) // 4)
    
    if len(data) < window or data['forward_returns'].std() == 0:
        # Fallback for short data or zero volatility
        return np.full(len(data), 1.0) 

    # Calculate rolling standard deviation (volatility)
    volatility = data['forward_returns'].rolling(window).std().fillna(method='bfill').values
    
    # RP allocates weights inversely to volatility: P_i proportional to 1/sigma_i
    # Calculate the inverse volatility weight, using EPS to prevent division by zero
    inv_volatility = 1 / (volatility + EPS)
    
    # Normalize the positions such that their mean is 1.0 (to maintain scale with MVO/Sharpe)
    # np.nanmean is used for robustness against NaNs in early rolling periods
    normalized_weights = inv_volatility / np.nanmean(inv_volatility)
    
    # Apply to positions and clip to bounds [MIN_POSITION, MAX_POSITION]
    positions = np.clip(normalized_weights, MIN_POSITION, MAX_POSITION)
    
    # Handle NaNs
    positions = np.nan_to_num(positions, nan=1.0) 
    
    return positions


**CAPM Alpha-based Signal Strategy**

Objective Function:

The strategy's objective is to set the market position proportional to the historically realized Alpha signal (i.e., the market's excess return over the risk-free rate).

Position t = 1.0 + (Rolling Excess Return t * Scaling Factor)

Where:
* Position t = Allocation to the risky asset at time t.
* Rolling Excess Return t = Rolling mean of (Market Return - Risk-Free Rate).
* Scaling Factor = A constant used to translate the small daily excess return into a position size.
* 1.0 = A base market-neutral position, which is adjusted up or down by the signal.



***Characteristics:***
* Factor-Based: This is a factor-based strategy where the factor is historical outperformance (Alpha).
* Simple and Quick: It uses a straightforward rolling mean calculation, avoiding the complexity and time of numerical optimization.
* Momentum/Trend-Following: The strategy implicitly acts as a medium-term momentum/trend-following signal by investing more when the market has recently delivered positive excess returns.
* Risk-Free Benchmark: The signal is explicitly benchmarked against the risk-free rate, aligning with the core concept of CAPM.
* Requires Tuning: The Scaling Factor is a critical hyperparameter that must be tuned to keep the position within sensible bounds.


***When it works best:***
* Persistent Trends: Markets where historical market outperformance (positive alpha) is likely to continue into the future (i.e., when momentum is strong).
* Benchmark Exploitation: Environments where the market is consistently outperforming the risk-free rate (cash) and that trend is expected to persist.
* Simplicity and Speed: When fast, low-latency strategy calculation is necessary, as it doesn't rely on iterative minimization solvers.
* Outperforming the "Hold 1.0" Baseline: When there are clear periods of positive alpha and clear periods of negative alpha, allowing the strategy to scale up and down effectively.

In [15]:
# Strategy 5: CAPM Alpha-based Signal Strategy 
def capm_alpha_optimization(data):
    """
    Implements a simple time-series strategy based on a rolling Alpha signal.
    
    In this single-asset context, the 'Alpha' signal is approximated by the rolling 
    mean of the market's excess return over the risk-free rate. Positive alpha suggests
    historical outperformance and leads to a higher position.
    """
    window = min(20, len(data) // 4)
    
    # Calculate rolling mean of excess return (Alpha-like signal)
    market_excess = data['forward_returns'] - data['risk_free_rate']
    alpha_signal = market_excess.rolling(window).mean().fillna(method='bfill').values
    
    # Scale the position based on the Alpha signal
    # Base position is 1.0 (market neutral), scaled by the alpha signal times a factor (e.g., 50.0)
    positions = 1.0 + (alpha_signal * 50.0) 
    
    # Clip to bounds
    positions = np.clip(positions, MIN_POSITION, MAX_POSITION)
    
    return positions


**Sortino Ratio Maximization Strategy**

Objective Function:

Maximize: Sortino Ratio = (Portfolio Mean Return - Rf)/Downside Deviation}

Where:
* Downside Deviation is the standard deviation of only the returns that fall below the target return.
* Rf is the Ris Free-Rate

***Characteristics:***

* Asymmetric Risk Focus: The strategy explicitly distinguishes between "good" volatility (upside movement) and "bad" volatility (downside movement). It penalizes only losses.
* Risk Metric: Replaces total volatility (used in the Sharpe Ratio and Mean-Variance) with Downside Deviation, leading to a more intuitive measure of "bad" risk.
* Non-Linear Optimization: Requires a numerical solver (like the existing SLSQP used for Sharpe Maximization) to find the optimal time series of positions.
* Allocation Goal: Seeks to capture upward market moves while strongly suppressing drawdowns.

***When it works best:***

* Risk-Averse Investors: Ideal for clients or competitions (like the one in your notebook) where drawdown control and capital preservation are paramount.
* Non-Normal Distributions: When asset returns are negatively skewed (have frequent, large losses), the Sortino Ratio is a more robust performance measure than the Sharpe Ratio.
* Long-Term Strategy: Suitable for strategies focusing on compound growth, as minimizing the depth and frequency of negative returns is key to long-term compounding.
* Downside Protection: When the primary goal is to minimize the probability of returns falling below a set benchmark, such as the risk-free rate.

In [16]:
# Strategy 6: Sortino Ratio Maximization
def sortino_ratio_optimization(data):
    """
    Directly maximize the Sortino Ratio metric, focusing on reducing downside risk.
    
    Uses multiple random starting points (x0) and a non-linear solver (SLSQP).
    """
    n_days = len(data)
    
    def objective(positions):
        # Objective: Maximize Sortino Ratio. Negate it because we use 'minimize'.
        return -calculate_sortino_ratio(data, positions)
    
    best_result = None
    best_score = float('-inf')
    
    # Test multiple initial guesses to avoid getting stuck in local minima/maxima
    initial_guesses = [
        np.full(n_days, 0.5),  # Moderate
        np.full(n_days, 1.0),  # Market neutral
        np.random.uniform(0.3, 0.7, n_days),  # Random moderate
        np.linspace(0.2, 0.8, n_days),  # Gradual increase
    ]
    
    bounds = Bounds(MIN_POSITION, MAX_POSITION)
    
    for x0 in initial_guesses:
        try:
            # Optimization using SLSQP
            result = minimize(
                objective,
                x0,
                method='SLSQP',
                bounds=bounds,
                options={'maxiter': 1000, 'ftol': 1e-8}
            )
            
            # Check if this run is successful and better than the current best
            if result.success and -result.fun > best_score:
                best_score = -result.fun
                best_result = result
        except Exception as e:
            # Fail silently on initial guess failure
            continue
    
    if best_result is not None:
        # The result 'x' array contains the optimized positions
        return np.clip(best_result.x, MIN_POSITION, MAX_POSITION)
    else:
        # Fallback to a conservative position if all optimizations fail
        return np.full(n_days, 0.5)


The `evaluate_strategies` function, which serves as the main engine for the backtesting and selection phase of the portfolio optimization system. Its primary role is to test five different trading strategies on historical data, measure their performance using the Adjusted Sharpe Ratio, and select the best one.

In [17]:
# ============================================================================
# STRATEGY EVALUATION AND SELECTION
# ============================================================================

def evaluate_strategies(train_data):
    """
    Run all defined optimization strategies on the historical lookback window 
    and select the best performing one based on the Adjusted Sharpe Ratio.
    """
    # Use only the most recent data for optimization (rolling window approach)
    lookback = min(LOOKBACK_WINDOW, len(train_data))
    recent_data = train_data.iloc[-lookback:].copy()
    
    # Feature creation is run, though not directly used by the current optimization objectives
    recent_data = create_features(recent_data)
    
    # Calculate market metrics for context/reporting
    market_vol = recent_data['forward_returns'].std() * np.sqrt(252) * 100
    market_return = recent_data['forward_returns'].mean() * 252 * 100
    
    print("=" * 80)
    print("PORTFOLIO OPTIMIZATION STRATEGY EVALUATION")
    print("=" * 80)
    print(f"Using {lookback} days of historical data")
    print(f"Market volatility: {market_vol:.2f}%\nAverage market return: {market_return:.2f}%")
    print("=" * 80)
    
    strategies = {
        'Mean-Variance': mean_variance_optimization,
        'Minimum Variance': minimum_variance_optimization,
        'Sharpe Maximization': sharpe_ratio_optimization,
        'Risk Parity': risk_parity_optimization, 
        'CAPM Alpha Signal': capm_alpha_optimization,
        'Sortino Maximization': sortino_ratio_optimization
    }
    
    results = []
    
    for name, strategy_func in strategies.items():
        print(f"\nOptimizing {name} strategy...")
        
        try:
            # Execute the optimization
            positions = strategy_func(recent_data)
            
            # Evaluate the optimized positions using the competition metric
            sharpe = calculate_adjusted_sharpe(recent_data, positions)
            
            # Calculate additional performance metrics for the leaderboard
            portfolio_returns = (
                recent_data['risk_free_rate'].values * (1 - positions) +
                positions * recent_data['forward_returns'].values
            )
            
            avg_position = np.mean(positions)
            portfolio_vol = np.std(portfolio_returns) * np.sqrt(252) * 100
            
            results.append({
                'Strategy': name,
                'Adjusted Sharpe': sharpe,
                'Avg Position': avg_position,
                'Portfolio Vol (%)': portfolio_vol,
                'Positions': positions 
            })
            
            print(f"  ✓ Adjusted Sharpe: {sharpe:.4f}")
            print(f"  ✓ Average Position: {avg_position:.4f}")
            print(f"  ✓ Portfolio Volatility: {portfolio_vol:.2f}%")
            
        except Exception as e:
            print(f"  ✗ Failed: {str(e)}")
            results.append({
                'Strategy': name,
                'Adjusted Sharpe': -999,
                'Avg Position': 0,
                'Portfolio Vol (%)': 0,
                'Positions': np.full(lookback, 0.5)
            })
    
    # Create and display the performance leaderboard
    leaderboard = pd.DataFrame(results)
    leaderboard = leaderboard.sort_values('Adjusted Sharpe', ascending=False)
    
    print("\n" + "=" * 80)
    print("STRATEGY LEADERBOARD")
    print("=" * 80)
    print(leaderboard[['Strategy', 'Adjusted Sharpe', 'Avg Position', 'Portfolio Vol (%)']].to_string(index=False))
    print("=" * 80)
    
    # Select the overall best strategy
    best_strategy = leaderboard.iloc[0]
    print(f"\n🏆 WINNER: {best_strategy['Strategy']} with Sharpe {best_strategy['Adjusted Sharpe']:.4f}")
    
    # Return the optimal positions for the best strategy
    return best_strategy['Positions'], leaderboard


In [18]:
# ============================================================================
# INFERENCE SETUP
# ============================================================================

# Global variables to store the pre-calculated optimal positions and track progress
optimal_positions = None # Will store the 'Positions' array from the winning strategy
position_counter = 0     # Tracks the current day in the test set

def predict(test: pl.DataFrame) -> float:
    """
    Prediction function for the competition inference server (daily prediction).
    
    This function simulates the deployment phase where the optimal positions 
    calculated from the historical lookback window are executed sequentially.
    """
    global position_counter, optimal_positions
    
    # Check if there's a pre-calculated position for the current step
    if position_counter < len(optimal_positions):
        position = float(optimal_positions[position_counter])
    else:
        # If the test set is longer than the lookback window, use the last calculated position
        position = float(optimal_positions[-1])
    
    # Ensure position is within the defined bounds [0.0, 2.0]
    position = np.clip(position, MIN_POSITION, MAX_POSITION)
    
    print(f"[Step {position_counter}] Position: {position:.6f}")
    position_counter += 1
    
    return position # The required output: the position size for the current day



In [19]:
# ============================================================================
# MAIN EXECUTION
# ============================================================================
    
# Run strategy evaluation and get best positions
print("\nRunning portfolio optimization strategies...")
# This call calculates the 'optimal_positions' which are used in the 'predict' function
optimal_positions, leaderboard = evaluate_strategies(train)
    
    
print("\n✅ Execution complete!")


Running portfolio optimization strategies...
PORTFOLIO OPTIMIZATION STRATEGY EVALUATION
Using 180 days of historical data
Market volatility: 17.73%
Average market return: 8.44%

Optimizing Mean-Variance strategy...
  ✓ Adjusted Sharpe: 10.0203
  ✓ Average Position: 1.0667
  ✓ Portfolio Volatility: 19.81%

Optimizing Minimum Variance strategy...
  ✓ Adjusted Sharpe: 0.3690
  ✓ Average Position: 0.3000
  ✓ Portfolio Volatility: 5.28%

Optimizing Sharpe Maximization strategy...
  ✓ Adjusted Sharpe: 16.9171
  ✓ Average Position: 0.0978
  ✓ Portfolio Volatility: 0.26%

Optimizing Risk Parity strategy...
  ✓ Adjusted Sharpe: 0.2861
  ✓ Average Position: 0.9966
  ✓ Portfolio Volatility: 13.86%

Optimizing CAPM Alpha Signal strategy...
  ✓ Adjusted Sharpe: 0.6064
  ✓ Average Position: 1.0010
  ✓ Portfolio Volatility: 16.26%

Optimizing Sortino Maximization strategy...
  ✓ Adjusted Sharpe: 10.0203
  ✓ Average Position: 1.0667
  ✓ Portfolio Volatility: 19.81%

STRATEGY LEADERBOARD
            S

In [20]:
# Setup inference server
print("\nInitializing inference server...")
inference_server = kaggle_evaluation.default_inference_server.DefaultInferenceServer(predict)
    
# Run server
if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
# Competition environment
    inference_server.serve()
else:
# Local testing
    print("Running local gateway for testing...")
    inference_server.run_local_gateway(('/kaggle/input/hull-tactical-market-prediction/',))
    


Initializing inference server...
Running local gateway for testing...
[Step 0] Position: 0.000000
[Step 1] Position: 0.000000
[Step 2] Position: 0.043087
[Step 3] Position: 0.000000
[Step 4] Position: 0.000000
[Step 5] Position: 0.080030
[Step 6] Position: 0.000000
[Step 7] Position: 0.000000
[Step 8] Position: 0.000000
[Step 9] Position: 0.027716
